# FX option pricing with Vanna-Volga method

## 1. Introduction

The foreign-exchange options market is one of the largest and most liquid OTC derivatives markets in the world. Most trading is over the counter and lightly regulated (a fraction is traded on exchanges like CME - Chicago Mercantile Exchange).

This market has developed its own way to quote options, which differs significantly from other markets. In addition to that, the precise meaning of the broker quotes depends on the details of the contract: for example, there are at least four different definitions for at-the-money strike (resp., “spot”, “forward”, “delta neutral”, “50 delta call”).<br>
In this market, brokers usually quote volatilities in terms of delta instead of quoting volailities in terms of strike values or giving directly the price of the instrument. Market participants considers this input as it is the only parameter in Black-Scholes to correct in order to have an accurate price. The Black–Scholes theoretical value is used only as a reference quotation, to ensure that the involved counterparties are speaking of the same option.<br>
Delta represents the derivative of the price of an option with respect to the spot. Implement a delta hedge strategy will make one’s position insensitive to small FX spot movements.

Common quotes for volatility are:
- **ATM volatility**: the value from the smile curve where the strike is such that the Delta of the call equals, in absolute value, that of the put (this strike is termed ATM “straddle” or ATM “delta neutral”).
- **25-delta Risk Reversal**: the volatility at the strikes $K_c$, $K_p$ that satisfy $\Delta Call(K_c,\ \sigma(K_c)) = 0.25$ and $\Delta Put(K_p,\ \sigma(K_p)) = -0.25$ respectively.
    - *Risk Reversal* is a measure of the skew in the demand for out-of-the-money (OTM) options at high strikes compared to low strike and can be interpreted as the market view of the most likely direction of the spot movement over the next maturity date.
    - $Risk\ Reversal = Call\ implied\ volatility - Put\ imply\ volatility$ with same deltas
- **25-delta Butterfly** : 
    - *Butterfly* is a measure of the average demand for OTM options at low strikes and high strikes compared to those near the forward level and can be interpreted as the market view of the likelihood of larger moves in the spot price over the next maturity.
    - $Butterfly = 0.5 \times (call - put) - ATM$

Below is the the relative positioning of different deltas within a stylized volatility smile (source: [FX Derivatives Trader School by Giles Jewitt](https://www.oreilly.com/library/view/fx-derivatives-trader/9781118967454/c12.xhtml))
![title](https://www.oreilly.com/library/view/fx-derivatives-trader/9781118967454/images/c12ex002.jpg)

## 2. Vanna-Volga method

### 2.1. The limitations of the Black-Scholes model

The Black-Scholes model is one of the most popular models for options pricing despite having some limitations, based on several unrealistic assumptions, that render the price inaccurate and are problematic in our case such as:
- Assuming constant value for risk free rate of return over the option duration.
- Assuming constant value for volatility over the option duration.
    - This comes down to consider that the volatility smile is flat, which is not true as you can see below (EUR USD volatility smiles as of February, 12th 2019).
![alt text](EUR USD smiles.png "EUR USD")

The most famous defect of the Black–Scholes model is the (wrong) assumption that the volatility is constant throughout the lifetime of the option. <br>
A foreseeable manner to calculate the relevant price Black-Scholes prices is to find the accurate volatility through the volatility surface.

Others models should assume that interest rates and the FX spot volatility follow stochastic processes that are coupled to the one of the spot, the choice of the stochastic process depending on empirical observations. But we known that for long-dated options the effect of the interest rate volatility can become as significant as that of the FX spot volatility, therefore a mis-calibration can lead to significant mis-pricing. <br>
Stochastic volatility models are computationally demanding and in most cases require a delicate calibration procedure in order to find the value of parameters that allow the model reproduce the market dynamics.

### 2.2. Vega, Vanna & Volga

The **vega** is the sensitivity of an option to the changes in the implied volatility for a maturity equal to its stopping time. It has a bell shape and generally decreases with time. The ATM Vega is the peak and it decreases more and more for deep in-the-money and out-of-the-money options.
\begin{align}
\nu = \frac{\delta F}{\delta \sigma}
\end{align}
where F is the derivative security and $\sigma$ the implied volatility
- For example, if volatility is 9% and the vega is 0.5, the option will pick up 50 cents when the volatility raised by one percentage up to 10%.

A highly positive or negative Vega implies that the portfolio is very sensitive to the small changes of volatility.<br>
If the value of Vega is close to zero, it suggests that the volatility has little impact on the value of the portfolio. <br>
**The Black-Scholes model cannot take care of the sensitivity of Vega** because a Vega-neutral position is subject to changes of spot and volatility. Therefore, we need to know the sensitivity of Vega to the changes in spot (vanna) and implied volatility (volga).
<br><br><br>
**Vanna** is the rate at which the delta and vega of an option will change as the volatility and price of the underlying security change respectively. It can be defined in three different ways:
- the change of Vega V with respect to the change in underlying price S.
\begin{align}
vanna = \frac{\delta V}{\delta S}
\end{align}
- the sensitivity of Delta $\Delta$ with respect to the change in volatility
\begin{align}
vanna = \frac{\delta \Delta}{\delta \sigma}
\end{align}
- the sensitivity of option value P with respect to a joint movement in volatility $\sigma$ and the underlying price S
\begin{align}
vanna = \frac{\delta^2 P}{\delta \sigma \delta S}
\end{align}

In Black-Scholes model, the vanna of simple option is:
\begin{align}
vanna = e^{-qt} \sqrt{T - t} N'(d1) \frac{d2}{\sigma}
\end{align}

Vanna can be useful to watch when a trader want to be delta neutral or vega neutral. This metric can be useful when you need to determine whether your options portfolio is net long/short calls/puts while holding multiple positions. Points to consider:
- long call positions and short put options have positive vanna.
- short call positions and long put options have negative vanna.
    - because an increase in volatility will increase the change of an option that is moving towards the money.
<br><br>

**Volga** or volatility Gamma represents the sensitivity of vega with respect to the change in volatility. A positive value for vomma indicates that a percentage point increase in volatility will result in an increased option value which is demonstrated by vega’s convexity, meaning that the option with high volga can benifit from volatility of volatility. It can be defined in two different ways:
- the change in vega with respect to a change in volatility $\sigma$
\begin{align}
volga = \frac{\delta vega}{\delta \sigma}
\end{align}
- the second derivative of option value P with respect to changes in volatility $\sigma$
\begin{align}
vanna = \frac{\delta^2 P}{\delta \sigma}
\end{align}

In Black-Scholes model, the volga of simple option is:
\begin{align}
volga = e^{-qt} \sqrt{T - t} N'(d1) \frac{d1 \dot d2}{\sigma}
\end{align}


Points to consider:
- volga is positive for options not in the money, and generally increases as the option gets deeper out-of-the-money. 
    - a positive volga means that a position will become long vega as implied volatility increases and vice-versa.
- it decreases when vega drops.

### 2.3. Vanna-Volga option pricing

Vanna‐Volga is a popular method for interpolation/extrapolation of volatility smiles. This method consider that the price of an option formulated with Black-Scholes formula can be corrected by adding the hedging of three main risks associated to the volatility of the option presented above: the vega, the vanna and the volga. Therefore, the hedging portfolio contains the three following strategies based on three main volatility quotes that are available for a given maturity:
- the delta-neutral straddle:
\begin{align} ATM = 0.5 \times Straddle(K_{ATM}) \end{align}
- the risk reversal:
\begin{align} Risk\ Reversal = Call(K_c, \sigma(K_c)) - Put(K_c, \sigma(K_c)) \end{align}
- the vega-weighted butterfly:
\begin{align} Butterfly = 0.5 \times Strangle(K_c, K_p) - 0.5 \times Straddle(K_{ATM}) \end{align}

Besides being intuitive and easy to implement, this procedure has a clear financial interpretation, which further supports its use in practice.
- First, it is an efficient tool for interpolating and extrapolating volatility for a given maturity while reproducing exactly the market quoted volatilities.
- Second, it can be employed in any market where at least three volatility quotes are available for a given maturity.
- Third, this method can derive implied volatilities for any options delta, particularly for those outside the basic range set.
- Fourth, this non-parametric method produces a consistent and complete smile with just three prices for each maturity.
- Fifth, it is supported by a clear financial rationale based on a hedging argument. 
- Finally, this method allows for the automatic calibration to the three input volatilities derived from market prices and acts as an explicit function of them.

![alt text](EUR USD surface.png "EUR USD")

## 3. Our FX option pricer

This pricer give the premium in EUR for EUR / USD european option.

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sct
from datetime import datetime

In [2]:
class FX_Option:
    
    def __init__(self, notionalCall, notionalPut, direction, expiry, spot, Rates, tToMat, impVol):
        self.notionalCall = notionalCall #call notional
        self.notionalPut = notionalPut #put notional
        self.direction = direction #direction for the 2 ccys
        self.expiry = expiry #settlement date
        self.spot = spot #fx spot 1 put_notional in call_notional_currency
        
        self.K_t = self.notionalCall / self.notionalPut #strike
        self.t = ((datetime.strptime(self.expiry, "%d/%m/%Y") - datetime.today().replace(hour = 0,
                minute = 0, second = 0, microsecond = 0)).days) / 365 #time to expiry from today (expressed in years)
        
        self.EURois = np.array(Rates["EUR OIS"]) #eur ois interest rate
        self.USDois = np.array(Rates["USD OIS"]) #usd ois interest rate
        self.F = np.array(Rates["EUR/USD Fwd"]) #EUR/USD forward rate
        
        self.tToMat = np.array(tToMat) #tenor expressed in years
        
        self.d25call = np.array(impVol["ATM"] + impVol["25D BF"] + 0.5 * impVol["25D RR"]) #
        self.d25put = np.array(impVol["ATM"] + impVol["25D BF"] - 0.5 * impVol["25D RR"]) #
        self.ATM = np.array(impVol["ATM"]) #
        self.d10call = np.array(impVol["ATM"] + impVol["10D BF"] + 0.5 * impVol["10D RR"]) #
        self.d10put = np.array(impVol["ATM"] + impVol["10D BF"] - 0.5 * impVol["10D RR"]) #
        
        self.r = np.interp(x = self.t, xp = self.tToMat, fp = self.USDois) #r_t rajouter si call=USD -> USD & inverse
        self.df = 100 / (100 + self.r) #discount factor call_notional_currency
        self.F_t = np.interp(x = self.t, xp = tToMat, fp = self.F) 
        
        self.r_d = -np.log(self.df) / self.t #domestic interest rates implied from discount factor
        self.r_e = self.r_d - np.log(self.F_t / self.spot) / self.t #foreign interest rates
        
        self.vPut = self.vInt(self.d25put) \
                    if (self.K_t > self.vInt(self.d25put) and self.K_t > self.vInt(self.d25call)) else \
                    self.vInt(self.d10put)
        self.vATM = self.vInt(self.ATM) \
                    if (self.K_t > self.vInt(self.ATM) and self.K_t > self.vInt(self.ATM)) else \
                    self.vInt(self.ATM)
        self.vCall = self.vInt(self.d25call) \
                    if (self.K_t > self.vInt(self.d25put) and self.K_t > self.vInt(self.d25call)) else \
                    self.vInt(self.d10call)      
        self.kPut = self.relevantK(self.vPut, "Put", 0.25) \
                    if (self.K_t > self.vInt(self.d25put) and self.K_t > self.vInt(self.d25call)) else \
                    self.relevantK(self.vPut, "Put", 0.1)
        self.kATM = self.relevantK(self.vATM, "ATM", 0.25) \
                    if (self.K_t > self.vInt(self.d25put) and self.K_t > self.vInt(self.d25call)) else \
                    self.relevantK(self.vATM, "ATM", 0.1)
        self.kCall = self.relevantK(self.vCall, "Call", 0.25) \
                    if (self.K_t > self.vInt(self.d25put) and self.K_t > self.vInt(self.d25call)) else \
                    self.relevantK(self.vCall, "Call", 0.1)
        
        #first approximation of implied volatility
        self.firstVol = self.firstOrderVol(self.kPut,self.kATM,self.kCall, self.K_t, self.vPut, self.vATM, self.vCall)
        
        #second approximation of implied volatility
        self.secondVol = self.secondOrderVol(self.kPut,self.kATM,self.kCall,self.K_t,self.vPut, self.vATM, self.vCall)
        
        #switch between EUR call & EUR put
        self.z = - 1 if self.direction.lower().find("eur put") != -1 else 1 #if eur put -> -1 , eur call -> 1
        
        #notional in eur
        self.EURnot = notionalCall if self.direction.lower().find("eur call") != -1 else notionalPut
        
        #option price
        ##price in currency
        self.p = self.EURnot * self.df * (self.z * self.F_t * \
                    sct.norm.cdf(self.z * self.d1(self.F_t, self.K_t, self.secondVol, self.t)) - self.z * self.K_t * \
                    sct.norm.cdf(self.z * self.d2(self.F_t, self.K_t, self.secondVol, self.t))) / self.spot
    
 
    ##price in % of notional
    def pricePct(self):
        return print("""FX option: {0:<20s}\nPrice: {1:2.4f}%""".format(self.direction , \
                                                                        (self.p / self.notionalPut) * 100))
    ##price in currency
    def price(self):
        return print("""FX option: {0:<20s}\nPrice: {1:,.2f} EUR""".format(self.direction, self.p))
    ##Greeks summary
    def Greeks(self):
        return print("FX option: {0:<20s}\nDelta: {1:1.4f} \t Gamma: {2:1.4f} \t Theta: {3:1.4f} \t Vega: {4:1.4f}\
            \nRho: {5:1.4f} \t Vanna: {6:1.4f} \t Volga: {7:1.4f}".format(self.direction, self.delta(), self.gamma(),\
                self.theta(), self.vega(), self.rho(),self.vanna(), self.volga()))
    
    
    #find the volatility with linear interpolation
    def vInt(self, vecVol):
        return np.interp(x = self.t, xp = self.tToMat, fp = vecVol)

    
    #calculate the strike
    def relevantK(self, v, pos, d):
        if pos == "ATM":
            k = self.spot * np.exp((self.r_d - self.r_e + 0.5 * (v / 100) ** 2) * self.t)
        elif pos == "Call":
            k = self.spot * np.exp(-sct.norm.ppf(d * np.exp(self.r_e * self.t)) * (v / 100) * np.sqrt(self.t) + \
                                   (self.r_d - self.r_e + 0.5 * (v / 100) ** 2) * self.t)
        elif pos == "Put":
            k = self.spot * np.exp(sct.norm.ppf(d * np.exp(self.r_e * self.t)) * (v / 100) * np.sqrt(self.t) + \
                                   (self.r_d - self.r_e + 0.5 * (v / 100) ** 2) * self.t)
        return k

    
    #calculate the 2nd order approximation of volatility
    def firstOrderVol(self, kPut, kATM, kCall, k, volPut, volATM, volCall):
        ch1 = self.chi1(kPut, kATM, kCall, k)
        ch2 = self.chi2(kPut, kATM, kCall, k)
        ch3 = self.chi3(kPut, kATM, kCall, k)
        return np.float64(ch1 * volPut + ch2 * volATM + ch3 * volCall) #eventuellement entourer par np.float64

    #used to calculate the 1st order approximation of volatility & D2
    def chi1(self, kPut, kATM, kCall, k):
        return (np.log(kATM / k) * np.log(kCall / k)) / (np.log(kATM / kPut) * np.log(kCall / kPut))
        
    def chi2(self, kPut, kATM, kCall, k):
        return (np.log(k / kPut) * np.log(kCall / k)) / (np.log(kATM / kPut) * np.log(kCall / kATM))
    
    def chi3(self, kPut, kATM, kCall, k):
        return (np.log(k / kPut) * np.log(k / kATM)) / (np.log(kCall / kPut) * np.log(kCall / kATM))
    
    
    #calculate the 2nd order approximation of volatility
    def secondOrderVol(self, kPut, kATM, kCall, k, volPut, volATM, volCall):
        return self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall) + \
            ((-self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall) + \
            np.sqrt((self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall) ** 2) + \
            self.d1(self.F_t, self.K_t, self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall), self.t) * \
            self.d2(self.F_t, self.K_t, self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall), self.t) * \
            (2 * self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall) * \
            self.D1(kPut, kATM, kCall, k, volPut, volATM, volCall) * \
            self.D2(kPut, kATM, kCall, k, volPut, volATM, volCall)))) / \
            (self.d1(self.F_t, self.K_t, self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall), self.t) * \
            self.d2(self.F_t, self.K_t, self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall), self.t)))

    
    #intermediate calculations for 2nd order approximation of volatility
    def d1(self, F,K,sigma,t):
        return (np.log(F / K) + (0.5 * (sigma / 100) ** 2) * t) / ((sigma / 100) * np.sqrt(t))
    
    def d2(self, F,K,sigma,t):
        return self.d1(F,K,sigma,t) - (sigma / 100) * np.sqrt(t)
    
    def D1(self, kPut, kATM, kCall, k, volPut, volATM, volCall):
        return self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall) - volATM
   
    def D2(self, kPut, kATM, kCall, k, volPut, volATM, volCall):
        return self.chi1(kPut, kATM, kCall, k) * self.d1(self.F_t, kPut, volPut, self.t) * \
                self.d2(self.F_t, kPut, volPut, self.t) * ((volPut - \
                    self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall)) ** 2) * \
            self.chi2(kPut, kATM, kCall, k) * self.d1(self.F_t, kATM, volATM, self.t) * \
                self.d2(self.F_t, kATM, volATM, self.t) * ((volATM - \
                    self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall)) ** 2) * \
            self.chi3(kPut, kATM, kCall, k) * self.d1(self.F_t, kCall, volCall, self.t) * \
                self.d2(self.F_t, kCall, volCall, self.t) * ((volCall - \
                    self.firstOrderVol(kPut, kATM, kCall, k, volPut, volATM, volCall)) ** 2) 
    
    #Greeks
    ##delta
    def delta(self):
        return self.z * self.df * sct.norm.cdf(self.z * self.d1(self.F_t, self.K_t, self.secondVol, self.t))
    
    #gamma
    def gamma(self):
        return sct.norm.pdf(self.d1(self.F_t, self.K_t, self.secondVol, self.t)) / \
                    (self.F_t * self.secondVol * np.sqrt(self.t))
    ##theta
    def theta(self):
        return (-(self.F_t * sct.norm.pdf(self.d1(self.F_t, self.K_t, self.secondVol, self.t)) * self.secondVol) * \
                (2 * np.sqrt(self.t))) + self.z * ((self.r_d - self.r_e) * self.F_t * \
                sct.norm.cdf(self.z * self.d1(self.F_t, self.K_t, self.secondVol, self.t)) - (self.r_d - self.r_e) * \
                self.K_t * self.df * sct.norm.cdf(self.z * self.d2(self.F_t, self.K_t, self.secondVol, self.t)))
    ##rho
    def rho(self):
        return (self.z * self.t * self.K_t * self.df * \
                sct.norm.cdf(self.z * self.d2(self.F_t, self.K_t, self.secondVol, self.t)))
    ##vega
    def vega(self):
        return (self.F_t * sct.norm.pdf(self.d1(self.F_t, self.K_t, self.secondVol, self.t)) * np.sqrt(self.t))
    ##vanna
    def vanna(self):
        return self.df * np.sqrt(self.t) * sct.norm.cdf(self.d1(self.F_t, self.K_t, self.secondVol, self.t)) * \
            (self.d2(self.F_t, self.K_t, self.secondVol, self.t) / self.secondVol)
    ##volga
    def volga(self):
        return self.df * np.sqrt(self.t) * sct.norm.cdf(self.d1(self.F_t, self.K_t, self.secondVol, self.t)) * \
            ((self.d1(self.F_t, self.K_t, self.secondVol, self.t) * self.d2(self.F_t, self.K_t, self.secondVol, \
                                                                            self.t)) / self.secondVol) 

In [3]:
FXSpot = float(pd.read_csv("EUR USD FX rate - 12 02 2018 - 17h34.csv", sep = ";").values)
FXSpot

1.1313

In [4]:
rates = pd.read_csv("EUR USD OIS rates - 12 02 2018 - 17h34.csv", sep = ";").iloc[::,1:]
timeToMaturity = [1 / t for t in [365, 52, 52 * 1/2, 52 * 1/3, 12, 12 * 1/2, 12 * 1/3, 12 * 1/4, 12 * 1/5,
                    12 * 1/6, 12 * 1/9, 1, 12 * 1/18,1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/10, 1/15, 1/20, 1/25, 1/30]]
rates = rates.interpolate(method = "linear", axis = 0)

In [5]:
implyVolTab = pd.read_csv("EUR USD Implied Volatilities - 12 02 2018 - 17h34.csv", sep = ";")

In [6]:
Put1 = FX_Option(11600000,10000000, "USD Call / EUR Put", "14/06/2020", FXSpot, rates, timeToMaturity, implyVolTab)
Put2 = FX_Option(12000000,10000000, "USD Call / EUR Put", "14/06/2020", FXSpot, rates, timeToMaturity, implyVolTab)

In [7]:
Put1.pricePct()
Put2.pricePct()

FX option: USD Call / EUR Put  
Price: 2.7041%
FX option: USD Call / EUR Put  
Price: 4.4382%


In [8]:
Put1.price()
Put2.price()

FX option: USD Call / EUR Put  
Price: 270,414.00 EUR
FX option: USD Call / EUR Put  
Price: 443,823.45 EUR


In [9]:
Put1.Greeks()
Put2.Greeks()

FX option: USD Call / EUR Put  
Delta: -0.3994 	 Gamma: 0.0384 	 Theta: -7.8591 	 Vega: 0.5268            
Rho: -0.6635 	 Vanna: 0.0128 	 Volga: 0.0030
FX option: USD Call / EUR Put  
Delta: -0.5542 	 Gamma: 0.0396 	 Theta: -7.7992 	 Vega: 0.5332            
Rho: -0.9309 	 Vanna: -0.0169 	 Volga: 0.0029


In [10]:
Call1 = FX_Option(11800000,10000000, "USD Put / EUR Call", "14/06/2020", FXSpot, rates, timeToMaturity, implyVolTab)
Call2 = FX_Option(12000000,10000000, "USD Put / EUR Call", "14/06/2020", FXSpot, rates, timeToMaturity, implyVolTab)

In [11]:
Call1.price()
Call2.price()

FX option: USD Put / EUR Call  
Price: 400,036.69 EUR
FX option: USD Put / EUR Call  
Price: 312,828.64 EUR


In [12]:
Call1.pricePct()
Call2.pricePct()

FX option: USD Put / EUR Call  
Price: 4.0004%
FX option: USD Put / EUR Call  
Price: 3.1283%


In [13]:
Call1.Greeks()
Call2.Greeks()

FX option: USD Put / EUR Call  
Delta: 0.5001 	 Gamma: 0.0399 	 Theta: -7.9682 	 Vega: 0.5407            
Rho: 0.7293 	 Vanna: -0.0043 	 Volga: -0.0001
FX option: USD Put / EUR Call  
Delta: 0.4224 	 Gamma: 0.0396 	 Theta: -7.7990 	 Vega: 0.5332            
Rho: 0.6199 	 Vanna: -0.0169 	 Volga: 0.0029


## Bibliography

Vanna Volga and Smile-consistent Implied Volatility Surface of Equity Index Option - Kun Huang<br>
The Vanna-Volga method for implied volatilities: tractability and robustness - Antonio Castagna & Fabio Mercurio <br>
Vanna-Volga Methods Applied to FX Derivatives: from theory to market practice - Frédéric Bossens, Grégory Rayée, Nikos S. Skantzos & Griselda Deelstra<br>
A Guide to FX Options Quoting Conventions - Dmitri Reiswich & Uwe Wystup